In [ ]:
import os
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Resolver, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
import tensorflow_model_analysis as tfma
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import Channel
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

In [ ]:
def create_pipeline(pipeline_name, pipeline_root, data_root, module_files, serving_model_dir):
    # Components
    output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        ])
    )
    example_gen = CsvExampleGen(input_base=data_root, output_config=output_config)

    statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
    schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
    example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
    transform = Transform(examples=example_gen.outputs['examples'], schema=schema_gen.outputs['schema'], module_file=os.path.abspath(module_files['transform_module']))

    trainer = Trainer(module_file=os.path.abspath(module_files['trainer_module']), examples=transform.outputs['transformed_examples'], transform_graph=transform.outputs['transform_graph'], schema=schema_gen.outputs['schema'], train_args=trainer_pb2.TrainArgs(splits=['train']), eval_args=trainer_pb2.EvalArgs(splits=['eval']))
    
    model_resolver = Resolver(strategy_class=LatestBlessedModelStrategy, model=Channel(type=Model), model_blessing=Channel(type=ModelBlessing)).with_id('Latest_blessed_model_resolver')

    eval_config = tfma.EvalConfig(model_specs=[tfma.ModelSpec(label_key='labels')], slicing_specs=[tfma.SlicingSpec()], metrics_specs=[tfma.MetricsSpec(metrics=[tfma.MetricConfig(class_name='ExampleCount'), tfma.MetricConfig(class_name='AUC'), tfma.MetricConfig(class_name='FalsePositives'), tfma.MetricConfig(class_name='TruePositives'), tfma.MetricConfig(class_name='FalseNegatives'), tfma.MetricConfig(class_name='TrueNegatives'), tfma.MetricConfig(class_name='BinaryAccuracy', threshold=tfma.MetricThreshold(value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.5}), change_threshold=tfma.GenericChangeThreshold(direction=tfma.MetricDirection.HIGHER_IS_BETTER, absolute={'value': 0.0001})))])])
    evaluator = Evaluator(examples=example_gen.outputs['examples'], model=trainer.outputs['model'], baseline_model=model_resolver.outputs['model'], eval_config=eval_config)

    pusher = Pusher(model=trainer.outputs['model'], model_blessing=evaluator.outputs['blessing'], push_destination=pusher_pb2.PushDestination(filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=serving_model_dir)))

    # Pipeline
    p = pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=[example_gen, statistics_gen, schema_gen, example_validator, transform, trainer, model_resolver, evaluator, pusher],
        metadata_connection_config=Metadata.sqlite_metadata_connection_config(os.path.join(pipeline_root, 'metadata.db'))
    )
    return p

In [ ]:
pipeline_name = 'ai_human_pipeline'
pipeline_root = os.path.join('pipelines', pipeline_name)
data_root = r"D:\Data"
module_files = {'transform_module': 'transform_module.py', 'trainer_module': 'trainer_module.py'}
serving_model_dir = os.path.join('serving_model', pipeline_name)

In [ ]:
my_pipeline = create_pipeline(pipeline_name, pipeline_root, data_root, module_files, serving_model_dir)
LocalDagRunner().run(my_pipeline)